<a href="https://colab.research.google.com/github/KaranJoseph/DemandForecasting_SCA/blob/main/InventoryManagement.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [8]:
# Install Pyomo and GLPK
!pip install -q pyomo
!apt-get install -y -qq glpk-utils #if GLPK is used
# !apt-get install -y -qq coinor-cbc #if cbc is used

In [9]:
import pandas as pd
df = pd.read_csv("https://raw.githubusercontent.com/KaranJoseph/DemandForecasting_SCA/main/Data/Output.csv")
df = df[["Item_ID", "Ship_Date", "Pred"]].dropna()

maxx = df["Pred"].max() + 50000 #Big M constraint

In [10]:
Co = 0.5 #Ordering Cost
Ch = 0.05 #Holding Cost per unit

In [11]:
from pyomo.environ import *

def optimizer(x, Co):
  """
    Parameters
    ----------
    x : numpy array
        Forecasted shipment quantities for 8 weeks
    Co : float 
        Order cost multiplier

    Returns
    -------
    model : pyomo object 
            Optimized inventory management model based on the obj and constraints of shortest path method (Integer Linear Programming Model)
  """
  Co = x.mean() * Co

  model = ConcreteModel()
  #Variables 
  model.y = Var(list(range(1, len(x)+1)), within=Binary) # Order yes or no
  model.s = Var(list(range(1, len(x)+1)), within=NonNegativeReals) # Inventory at period i
  model.q = Var(list(range(1, len(x)+1)), within=NonNegativeReals) # Order Quantity at period i

  #Objective
  obj1 = 0
  obj2 = 0
  for i in range(1, len(x)+1):
    obj1 += model.y[i]
    obj2 += model.s[i]
  model.OBJ = Objective(sense=minimize, expr = Co*obj1 + Ch*obj2)

  #Constraints
  model.order1 = Constraint(expr = model.q[1] <= model.y[1]*maxx)
  model.order2 = Constraint(expr = model.q[2] <= model.y[2]*maxx)
  model.order3 = Constraint(expr = model.q[3] <= model.y[3]*maxx)
  model.order4 = Constraint(expr = model.q[4] <= model.y[4]*maxx)
  model.order5 = Constraint(expr = model.q[5] <= model.y[5]*maxx)
  model.order6 = Constraint(expr = model.q[6] <= model.y[6]*maxx)
  model.order7 = Constraint(expr = model.q[7] <= model.y[7]*maxx)
  model.order8 = Constraint(expr = model.q[8] <= model.y[8]*maxx)


  model.inv1 = Constraint(expr = model.s[1] - model.q[1] + x[0] == 0) #Initial Inventory = 0 (Assumption)
  model.inv2 = Constraint(expr = model.s[2] - model.s[1] - model.q[2] + x[1] == 0)
  model.inv3 = Constraint(expr = model.s[3] - model.s[2] - model.q[3] + x[2] == 0)
  model.inv4 = Constraint(expr = model.s[4] - model.s[3] - model.q[4] + x[3] == 0)
  model.inv5 = Constraint(expr = model.s[5] - model.s[4] - model.q[5] + x[4] == 0)
  model.inv6 = Constraint(expr = model.s[6] - model.s[5] - model.q[6] + x[5] == 0)
  model.inv7 = Constraint(expr = model.s[7] - model.s[6] - model.q[7] + x[6] == 0)
  model.inv8 = Constraint(expr = model.s[8] - model.s[7] - model.q[8] + x[7] == 0)

  #model.pprint()
  opt = SolverFactory('glpk')
  opt.solve(model) 
  #model.display()
  return model

In [12]:
df_im = pd.DataFrame()
cost = {}

for item in df["Item_ID"].unique(): #Iterate through each item_id and calls optimizer function to get the best IM solution at Item level
  temp = df[df["Item_ID"] == item].reset_index().drop("index", axis=1)
  model = optimizer(temp["Pred"].values, Co)
  order = []
  qty = []
  inv = []
  for i in range(1, temp.shape[0]+1):
    order.append(model.y[i].value)
    qty.append(model.q[i].value)
    inv.append(model.s[i].value)

  qty = pd.Series(qty, name="OrderQty")
  inv = pd.Series(inv, name="Inventory")
  order = pd.Series(order, name="Order(yes/no)")
  t = pd.concat([temp, qty, inv, order], axis=1)
  df_im = df_im.append(t)

  cost[item] = round(model.OBJ(),2)

In [13]:
s1 = pd.Series(cost.keys(), name="Item_ID")
s2 = pd.Series(cost.values(), name="Cost")
s = pd.concat([s1,s2], axis=1)

df_im = pd.merge(df_im, s, on="Item_ID", how="left")

In [14]:
from google.colab import files

df_im.rename({"Pred": "ForecastedDemand"},axis=1, inplace=True)
df_im.to_csv("Inventory.csv", index=False)
files.download("Inventory.csv")

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>